In [1]:
import pandas as pd
from shapely.wkt import loads

# generate usr
df_traj = pd.read_csv('data/traj.csv')
df_usr = pd.DataFrame({'usr_id': df_traj['entity_id'].drop_duplicates().sort_values()})
df_usr.to_csv('jump.usr', index=False)

# generate geo
def wkt_to_coordinates(wkt):
    geom = loads(wkt)
    return f"[{geom.x}, {geom.y}]"

df_node_split = pd.read_csv('data/node_split.csv')
df_geo = pd.DataFrame({
    'geo_id': df_node_split['id'],  # Inheriting the 'geo_id' from 'id'
    'type': 'Point',  # Setting 'type' as 'Point' for all rows
    'coordinates': df_node_split['WKT'].apply(wkt_to_coordinates)  # Extracting coordinates from 'WKT'
})
df_usr.to_csv('jump.geo', index=False)


In [16]:
# generate dyna
df_traj = pd.read_csv('data/traj.csv')
for name, group in df_traj.groupby('traj_id'):
    print(f"Traj ID: {name}")
    print(group)
    break

Traj ID: 0
    id                  time  entity_id  traj_id             coordinates  \
0    0  2013-10-08T17:45:00Z        254        0  [116.318726,40.009014]   
1    1  2013-10-08T17:46:45Z        254        0  [116.315102,40.004784]   
2    2  2013-10-08T17:47:39Z        254        0  [116.315018,40.002842]   
3    3  2013-10-08T17:49:26Z        254        0  [116.315041,39.998585]   
4    4  2013-10-08T17:51:15Z        254        0  [116.315605,39.992554]   
5    5  2013-10-08T17:52:09Z        254        0  [116.315735,39.987846]   
6    6  2013-10-08T17:54:55Z        254        0  [116.313332,39.985687]   
7    7  2013-10-08T17:55:48Z        254        0  [116.305832,39.985264]   
8    8  2013-10-08T17:56:45Z        254        0   [116.298225,39.98381]   
9    9  2013-10-08T17:57:39Z        254        0  [116.300819,39.977249]   
10  10  2013-10-08T17:58:35Z        254        0  [116.305977,39.970459]   
11  11  2013-10-08T17:59:27Z        254        0  [116.307533,39.963829]   
1